In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **ALGORITMO PRINCIPAL DE PRESENTACIÓN**
---


In [0]:
#Todas las librerias necesarias excepto las dos primeras que es el algoritmo de clasificacion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from google.colab.patches import cv2_imshow
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from google.colab import files
from skimage import exposure
from skimage import feature
import argparse #Parsear desde la linea de comandos
from imutils import paths
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import cv2
import glob
import re

In [0]:
all_images_train = [] #Contiene las imagenes sin el HOG
data = []             #Datos HOG de cada imagen
labels = []           #Datos de etiquetas que identifican cada imagen

images_train = glob.glob('/content/drive/My Drive/Colab Notebooks/cancer_images/mama_cancer_png/train' + '/*.png')

for i in images_train:
  if i.endswith("png"):
    all_images_train.append(i)

In [0]:
data.shape

(512, 367236)

In [0]:
def splitfilename(filename):
    sname=""
    sext=""
    i=filename.rfind(".")
    if(i!=0):
        n=len(filename)
        j=n-i-1
        sname=filename[76:i]
        #sext=filename[-j:]    
    return sname

def splitfilenametest(filename):
    sname=""
    sext=""
    i=filename.rfind(".")
    if(i!=0):
        n=len(filename)
        j=n-i-1
        sname=filename[75:i]
        #sext=filename[-j:]    
    return sname

In [0]:
for img in all_images_train:
  image = cv2.imread(img)
  name = splitfilename(img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  edged = imutils.auto_canny(gray)
  H = feature.hog(gray, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
  data.append(H)
  labels.append(name)

In [0]:
#Mostrar datos del HOG
for i in labels:
  print(i)

for i in data:
  print(i)

In [0]:
#Entrenamiento por medio de KKN
print("Training KNN clasiffier")
model = KNeighborsClassifier(n_neighbors=2, p=1)

print(model.fit(data, labels))
print('Accuracy is: {:.2f}'.format(model.score(data, labels)))

Training KNN clasiffier
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=1,
                     weights='uniform')


In [0]:
#Accuracy of training set with KKN
print('Accuracy is: {:.2f}'.format(model.score(data, labels)))

Accuracy is: 0.50


In [0]:
#Support vector machine classifier
clf = svm.SVC()
data = np.array(data)
labels = np.array(labels)

clf.fit(data, labels)
print('Accuracy is: {:.2f}'.format(clf.score(data, labels)))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy is: 1.00


In [0]:
all_images_test = []
images_test = glob.glob('/content/drive/My Drive/Colab Notebooks/cancer_images/mama_cancer_png/test' + '/*.png')

for i in images_test:
  if i.endswith("png"):
    all_images_test.append(i)

In [0]:
for i in all_images_test:
  print(i)

In [0]:
#Texto de título predeterminado { display-mode: "code" }
for img in all_images_test:
  image = cv2.imread(img)
  name = splitfilenametest(img)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  (h, hog_image) = feature.hog(gray, orientations=9, pixels_per_cell=(10, 10), cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualise=True)
  pred = model.predict(h.reshape(1, -1))[0]

  print(pred)
  print(type(pred))

  #Visualize HOG image
  #hog_image = exposure.rescale_intensity(hog_image, out_range=(0, 255))
  #hog_image = hog_image.astype("uint8")
  #cv2_imshow(hog_image)

  text = str(pred)
  #Show prediction
  cv2.putText(image, "Prediction: " + text, (5, 25), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 255, 0), 3)
  cv2_imshow(image)
  cv2.waitKey(0)

  cv2.putText(image, "Original: " + name, (5,70), cv2.FONT_HERSHEY_TRIPLEX, 0.8, (0, 255, 0), 3)
  cv2_imshow(image)
  cv2.waitKey(0)

# **EJECUCIÓN Y PRUEBA DE ALGORITMOS**
